In [36]:
import json

In [37]:
# Load the OpenAPI JSON file
with open("doc.json", "r") as f:
    data = json.load(f)


In [38]:
def extract_resource_name(path: str) -> str:
    parts = [part for part in path.split('/') if part]
    return parts[-1] if parts else ""

In [39]:
known_collections = ['users', '_stores']

In [40]:
def pb_field_from_schema(name, schema):
    t = schema.get("type")
    nullable = schema.get("nullable", False)
    max_length = schema.get("maxLength")
    minimum = schema.get("minimum")
    maximum = schema.get("maximum")
    enum = schema.get("enum")
    _required = ''
    if nullable == False:
        _required = ' , Required: true'
    # Skip ID
    if name == "id":
        return None
    
    if enum and isinstance(enum, (list)):
        return f'''&core.SelectField{{Name: "{name}", Values: []string{{"{'","'.join(enum)}"}}{_required}}}'''

    # Handle auto-dates
    if name == "createdAt":
        return f'&core.AutodateField{{Name: "createdAt", OnCreate: true}}'
    if name == "updatedAt":
        return f'&core.AutodateField{{Name: "updatedAt", OnUpdate: true}}'

    # Handle relation fields
    if name.endswith("Id"):
        target = f'{name[:-2]}s'  # strip 'Id'
        if target in known_collections or target in ['users', 'stores']:
            return f'&core.RelationField{{Name: "{name}", CollectionId: {target}Col.Id{_required}}}'
        
    # Handle date fields
    if name.endswith("At") or name.endswith('Date'):
        return f'&core.DateField{{Name: "{name}"{_required}}}'
    
    # Handle email fields
    if name.startswith("email"):
        return f'&core.EmailField{{Name: "{name}"{_required}}}'
    
    # Handle URL fields
    if name.find('link') != -1 or name.find('website') != -1:
        return f'&core.URLField{{Name: "{name}"{_required}}}'
    
    # Handle File fields
    if name.find('thumbnail') != -1 or name.find('logo') != -1:
        return f'&core.FileField{{Name: "{name}"{_required}}}'
    
    # Handle Editor fields
    if name.find('content') != -1 or name.find('content') != -1:
        return f'&core.EditorField{{Name: "{name}"{_required}}}'

    # Type mappings
    if t == "string":
        if max_length and max_length <= 2048:
            return f'&core.TextField{{Name: "{name}"{_required}}}'
        else:
            return f'&core.TextField{{Name: "{name}"{_required}}}'
    elif t == "boolean":
        return f'&core.BoolField{{Name: "{name}"{_required}}}'
    elif t == "integer" or t == "number":
        min_part = f', Min: types.Pointer[float64]({minimum})' if minimum is not None else ''
        max_part = f', Max: types.Pointer[float64]({maximum})' if maximum is not None else ''
        return f'&core.NumberField{{Name: "{name}"{min_part}{max_part}{_required}}}'
    return None

In [41]:
def extract_relation_fields(properties: dict) -> dict:
    """Return fields matching [name]Id as {field_name: collection_name}."""
    relations = {}
    for field in properties:
        if field.endswith("Id") and field != "id":
            collection = field[:-2]
            relations[field] = f'{collection}s'
    return relations

In [42]:
def generate_pb_store(collection:str, path:str, tag:str, description:str, name:str, properties: dict):
    relations = extract_relation_fields(properties)
    _rels = ""
    for rel in relations:
        if relations[rel] in known_collections or relations[rel] in ['users', 'stores']:
            _rels += f'{relations[rel]}Col, _ := app.FindCollectionByNameOrId("{relations[rel]}")\n'
    collection_var = collection + "Col"
    code = f'// Migration for collection: {path} {collection} - {tag} - {description}\n'
    code += """package migrations
import (
    "github.com/pocketbase/pocketbase/core"
    "github.com/pocketbase/pocketbase/migrations"
    "github.com/pocketbase/pocketbase/tools/types"
)

func init() {
migrations.Register(func(app core.App) error {\n"""
    code += f'\t\t{_rels}\n'
    code += f'\t\t{collection_var}, err := app.FindCollectionByNameOrId("{collection}")\n'
    code += f'\t\tif err != nil {{ {collection_var} = core.NewBaseCollection("{collection}") }}\n'
    code += f'\t\t{collection_var}.ListRule = types.Pointer("")\n'
    code += f'\t\t{collection_var}.ViewRule = types.Pointer("")\n'
    code += f'\t\t{collection_var}.Fields.Add(\n'
    field_lines = []
    for prop_name, prop_schema in properties.items():
        line = pb_field_from_schema(prop_name, prop_schema)
        if line:
            field_lines.append("\t\t\t" + line + ",")
    code += '\n'.join(field_lines) + '\n\t\t)\n'
    code += f'\t\tif err := app.Save({collection_var}); err != nil {{ return err }}\n'
    code += '\t\treturn nil\n'
    code += """\t}, nil)
}\n"""
    #print(f'auto/{name}.go')
    return code

In [43]:
# Access the 'paths' object
known_collections = ['users', '_stores']
paths = data.get("paths", {})

In [44]:
def run (save_path, crete_file):
    # Loop through each path and check for GET methods
    i = 1000
    for m in ["get", "post"]:
        for path, methods in paths.items():
            if path == '/':
                continue
            if m in methods:
                #print(methods)
                #print(f"{m.upper()} path found: {path}")
                # Try to access responses -> 200 -> content -> application/json -> schema -> properties
                try:
                    collection = extract_resource_name(path)
                    if collection in ['{id}']:
                        path = path.replace('/{id}', '')
                        collection = extract_resource_name(path)
                        #print('{id}:', path)
                    description = methods[m]["responses"]["200"]["description"]
                    tag = methods[m]["tags"][0]
                    path = path.replace('/{id}', '')
                    _name = '_'.join(path.split('/')).replace('_api', f'{i:08d}_collection')
                    if _name.find('admin') != -1 and paths.get(path.replace('/admin', '')) and _name.find('returns') != -1:
                        pass
                        #print(path, path.replace('/admin', ''))
                    if _name.find('admin') != -1 and (collection in known_collections or paths.get(path.replace('/admin', ''))):
                        continue
                    elif _name.find('admin') != -1:
                        _name = _name.replace('_admin', '')
                    if collection in ['me', '{id}', 'featured', 'megamenu', '{productId}', '{cartId}', ':cartId', 'create', 'delete', 'login', 'signup', 'save', 'my', 'refresh', 'list', 'url', 'toggle', 'void'] or collection.find('-') != -1:
                        continue
                    if collection in known_collections:
                        continue
                    name = _name
                    _type = methods[m]["responses"]["200"]["content"]["application/json"]["schema"]["type"]
                    _data = properties = methods[m]["responses"]["200"]["content"]["application/json"]["schema"]["properties"].get("data", {}).get("items", {}).get("properties", None)
                    if _type == "object" and not _data:
                        properties = methods[m]["responses"]["200"]["content"]["application/json"]["schema"]["properties"]
                    else:
                        properties = methods[m]["responses"]["200"]["content"]["application/json"]["schema"]["properties"]["data"]["items"]["properties"]
                    code = generate_pb_store(collection, path, tag, description, name, properties)
                    known_collections.append(collection)
                    if crete_file:
                        with open(f'{save_path}/{name}.go', mode = "w", encoding = "utf-8") as migrations_file:
                            migrations_file.write(code)
                            migrations_file.close()
                    i += 100
                except Exception as err:
                    print(str(err), path)
                    pass
                    #print(f"GET path {path} has no '200' response with JSON schema properties.")

In [45]:
run('migrations', False)

'200' /api/admin/products/create-qr-code
'properties' /api/coupons
'properties' /api/countries/all
'application/json' /api/invoice/download/:orderId
'type' /api/plugins
'application/json' /api/admin/download-vendor-invoice/:orderNo
'type' /api/stores/public
'type' /api/admin/sitemap
'type' /api/admin/imports
'type' /api/admin/import/retry
'type' /api/admin/import/:importNo
'200' /api/admin/domains/check-dns
'type' /api/admin/productsale
'type' /api/admin/productsale
'type' /api/admin/productsale/product
'200' /api/admin/products/create-rating-and-reviews
'200' /api/admin/products/bulk-status-update
'200' /api/admin/products/bulk-delete
'200' /api/blogs
'type' /api/admin/imports/:importNo/stop
'type' /api/upload
'type' /api/upload/multiple
'type' /api/admin/productsale


In [46]:
path = "/api/admin/variants"
properties = paths[path]["post"]["responses"]["200"]["content"]["application/json"]["schema"]["properties"]#["return"]["properties"]
code = generate_pb_store("variants", path, "Variants", "List of Variants", "variants", properties)
save_path = 'migrations'
i = 3150
name = '_'.join(path.split('/')).replace('_admin', '').replace('_api', f'{i:08d}_collection')
with open(f'{save_path}/{name}.go', mode = "w", encoding = "utf-8") as migrations_file:
    migrations_file.write(code)
    migrations_file.close()